In [35]:
import nltk
import pandas as pd
import numpy as np
import re
import codecs
import string


In [36]:
stopwords = nltk.corpus.stopwords.words('english')
wn=nltk.WordNetLemmatizer()

In [96]:
questions=pd.read_csv('C:/Users/Akshat/Downloads/socialmedia-disaster-tweets-DFE1.csv',engine='python')
#questions.columns=['text', 'choose_one', 'class_label']
questions.head()

,choose_one,choose_one_gold,text
0,Relevant,Relevant,Just happened a terrible car crash
1,Relevant,Relevant,Our Deeds are the Reason of this #earthquake M...
2,Relevant,Relevant,"Heard about #earthquake is different cities, s..."
3,Relevant,Relevant,"there is a forest fire at spot pond, geese are..."
4,Relevant,Relevant,Forest fire near La Ronge Sask. Canada


In [16]:
questions['choose_one'].unique()

array(['Relevant', 'Not Relevant', "Can't Decide"], dtype=object)

In [25]:
questions['choose_one_gold'].unique()

array([1, 2, 0], dtype=int64)

In [97]:
questions.loc[questions['choose_one']=='Relevant', 'choose_one_gold'] = 1
questions.loc[questions['choose_one']=='Not Relevant', 'choose_one_gold'] = 2
questions.loc[questions['choose_one']=="Can't Decide", 'choose_one_gold'] = 0

In [98]:
questions.columns=[ 'choose_one', 'class_label','text']

In [99]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

In [101]:
questions=standardize_text(questions,'text')

In [102]:
questions.head()

,choose_one,class_label,text
0,Relevant,1,just happened a terrible car crash
1,Relevant,1,our deeds are the reason of this earthquake m...
2,Relevant,1,"heard about earthquake is different cities, s..."
3,Relevant,1,"there is a forest fire at spot pond, geese are..."
4,Relevant,1,forest fire near la ronge sask canada


In [34]:
questions.groupby("class_label").count()

,choose_one,text
class_label,,
0,16,16
1,4673,4673
2,6187,6187


In [103]:
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [104]:
questions['clean'] = questions['text'].apply(lambda x: clean_text(x))

In [156]:
from  nltk.tokenize  import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

questions["tokens"] = questions["text"].apply(tokenizer.tokenize)
questions.head()

,choose_one,class_label,text,clean,tokens
0,Relevant,1,just happened a terrible car crash,"[happened, terrible, car, crash]","[just, happened, a, terrible, car, crash]"
1,Relevant,1,our deeds are the reason of this earthquake m...,"[deed, reason, earthquake, may, allah, forgive...","[our, deeds, are, the, reason, of, this, earth..."
2,Relevant,1,"heard about earthquake is different cities, s...","[heard, earthquake, different, city, stay, saf...","[heard, about, earthquake, is, different, citi..."
3,Relevant,1,"there is a forest fire at spot pond, geese are...","[forest, fire, spot, pond, goose, fleeing, acr...","[there, is, a, forest, fire, at, spot, pond, g..."
4,Relevant,1,forest fire near la ronge sask canada,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]"


In [159]:
questions.head()

,choose_one,class_label,text,clean,tokens
0,Relevant,1,just happened a terrible car crash,"[happened, terrible, car, crash]","[just, happened, a, terrible, car, crash]"
1,Relevant,1,our deeds are the reason of this earthquake m...,"[deed, reason, earthquake, may, allah, forgive...","[our, deeds, are, the, reason, of, this, earth..."
2,Relevant,1,"heard about earthquake is different cities, s...","[heard, earthquake, different, city, stay, saf...","[heard, about, earthquake, is, different, citi..."
3,Relevant,1,"there is a forest fire at spot pond, geese are...","[forest, fire, spot, pond, goose, fleeing, acr...","[there, is, a, forest, fire, at, spot, pond, g..."
4,Relevant,1,forest fire near la ronge sask canada,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]"


In [106]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [141]:
list_corpus = questions["clean"].tolist()
list_labels = questions["class_label"].tolist()

In [210]:
X_train, X_test, y_train, y_test = train_test_split(questions[["text"]], questions[["class_label"]], test_size=0.2,random_state=4)


In [211]:
#y_train = y_train['class_label'].astype(int)
y_train = list(y_train.class_label.values)

In [164]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['text'])
tfidf_train = tfidf_vect_fit.transform(X_train['text'])
tfidf_test = tfidf_vect_fit.transform(X_test['text'])

In [165]:
X_train_vect =pd.DataFrame(tfidf_train.toarray())
X_test_vect =pd.DataFrame(tfidf_test.toarray())

In [195]:
X_train_vect.shape

(8700, 14588)

In [94]:
# from sklearn.decomposition import PCA, TruncatedSVD
# import matplotlib.pyplot as plt
# import matplotlib.patches as mpatches


# def plot_LSA(test_data, test_labels, savepath="PCA_demo.csv", plot=True):
#         lsa = TruncatedSVD(n_components=1)
#         lsa.fit(test_data)
#         lsa_scores = lsa.transform(test_data)
#         color_mapper = {label:idx for idx,label in enumerate(set(test_labels))}
#         color_column = [color_mapper[label] for label in test_labels]
#         colors = ['orange','blue','blue']
#         if plot:
#             plt.scatter(lsa_scores[:,0], lsa_scores[:,1], s=8, alpha=.8, c=test_labels, cmap=matplotlib.colors.ListedColormap(colors))
#             red_patch = mpatches.Patch(color='orange', label='Irrelevant')
#             green_patch = mpatches.Patch(color='blue', label='Disaster')
#             plt.legend(handles=[red_patch, green_patch], prop={'size': 30})


# fig = plt.figure(figsize=(16, 16))          
# plot_LSA(tfidf_train, y_train)
# plt.show()

In [219]:
# y_train=np.array(y_train)
# y_train=y_train.reshape(8700,1)
#y_test=np.array(y_test)
#y_test=y_test.reshape(2176,1)

In [217]:
len(y_test)

2176

In [199]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()

In [213]:
clf_tfidf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', n_jobs=-1, random_state=40,
                               multi_class='multinomial')
clf_tfidf.fit(X_train_vect, (y_train))

y_predicted_tfidf = clf_tfidf.predict(X_test_vect)

In [244]:
type(y_predicted_tfidf[1])
type(y_test[1][0])

int

In [258]:
y_predicted_tfidf.shape

(2176,)

In [260]:
accuracy_score(y_test,y_predicted_tfidf)

ValueError: Classification metrics can't handle a mix of unknown and multiclass targets

In [259]:
# from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_predicted_tfidf))

from sklearn.metrics import classification_report
#print(classification_report(y_test, y_predicted_tfidf,labels=['0','1','2']))

ValueError: Classification metrics can't handle a mix of unknown and multiclass targets

In [236]:
from sklearn.metrics import accuracy_score

In [246]:
#accuracy_score(y_test, y_predicted_tfidf,normalize=False)

In [ ]:
clf_tfidf.im